### Load tensorflow

In [0]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [6]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [7]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data=data.drop(['date','symbol'],axis=1)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data=data.head(1000)

### Convert Float64 to Float32

In [0]:
data=data.astype('float32')

### Divide the data into train and test sets

In [0]:
X=data.drop('close',axis=1)

In [0]:
y=data['close']

In [0]:
from sklearn.model_selection import train_test_split


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)


In [16]:
X_train.shape

(700, 4)

In [17]:
y_train.shape

(700,)

In [0]:
y_train=y_train.values

In [19]:
y_train.shape

(700,)

### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
train_x = transformer.fit_transform(X_train)
test_x = transformer.fit_transform(X_test)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [26]:
for i in range(100):
    y_train=y_train.reshape(700,1)
    w, b = train(train_x, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 7653.4507
Current Loss on iteration 1 7333.09
Current Loss on iteration 2 7037.844
Current Loss on iteration 3 6765.7466
Current Loss on iteration 4 6514.982
Current Loss on iteration 5 6283.877
Current Loss on iteration 6 6070.89
Current Loss on iteration 7 5874.6016
Current Loss on iteration 8 5693.7026
Current Loss on iteration 9 5526.986
Current Loss on iteration 10 5373.34
Current Loss on iteration 11 5231.739
Current Loss on iteration 12 5101.24
Current Loss on iteration 13 4980.9727
Current Loss on iteration 14 4870.134
Current Loss on iteration 15 4767.986
Current Loss on iteration 16 4673.845
Current Loss on iteration 17 4587.084
Current Loss on iteration 18 4507.127
Current Loss on iteration 19 4433.438
Current Loss on iteration 20 4365.5244
Current Loss on iteration 21 4302.9375
Current Loss on iteration 22 4245.257
Current Loss on iteration 23 4192.0977
Current Loss on iteration 24 4143.1074
Current Loss on iteration 25 4097.9565
Current Loss on 

### Get the shapes and values of W and b

In [27]:
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.0757465e-02]
 [1.0656616e-02]
 [1.0835792e-02]
 [3.2731915e+01]]
Bias:
 [32.731915]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
model = tf.keras.models.Sequential()


In [0]:

model.add(tf.keras.layers.Dense(1,input_shape=(4,)))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='rmsprop',loss='mse')

In [0]:
X_train=X_train.values

In [0]:
train_X=X_train.reshape((700,4))

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(train_X,y_train,epochs=5)

Epoch 1/5
700/700 [==============================] - 0s 499us/sample - loss: 7828865517579.7031
Epoch 2/5
700/700 [==============================] - 0s 33us/sample - loss: 6947247766147.6562
Epoch 3/5
700/700 [==============================] - 0s 33us/sample - loss: 6273449097204.2969
Epoch 4/5
700/700 [==============================] - 0s 33us/sample - loss: 5759240153521.0059
Epoch 5/5
700/700 [==============================] - 0s 33us/sample - loss: 5241650471239.6797


### Execute the model

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data = pd.read_csv('./Iris.csv')

### Splitting the data into feature set and target set

In [36]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
X=data.drop(['Id','Species'],axis=1)
y=data['Species']

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le=LabelEncoder()

In [41]:
le.fit(data['Species'])

LabelEncoder()

In [0]:
data['y1']=le.transform(data['Species'])

In [0]:
y=data['y1']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(105, 4)
(105,)
(45, 4)
(45,)


In [45]:
X_train.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
118,7.7,2.6,6.9,2.3
18,5.7,3.8,1.7,0.3
4,5.0,3.6,1.4,0.2
45,4.8,3.0,1.4,0.3
59,5.2,2.7,3.9,1.4


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
X_train=X_train.values

In [0]:
X_test=X_test.values

In [48]:
y_test.shape

(45,)

In [0]:
y_test=y_test.values

In [0]:
y_train=y_train.values

In [0]:
trainY = tf.keras.utils.to_categorical(y_train, num_classes=3)
testY = tf.keras.utils.to_categorical(y_test, num_classes=3)

In [52]:
trainY.shape

(105, 3)

### Divide the dataset into Training and test (70:30)

In [0]:
#Splitting is already done

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
model = tf.keras.models.Sequential()



#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Fitting the model and predicting 

In [54]:
model.fit(X_train,trainY,          
          validation_data=(X_test,testY),
          epochs=5,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
105/105 [==============================] - 0s 4ms/sample - loss: 1.1528 - acc: 0.3048 - val_loss: 1.1247 - val_acc: 0.4000
Epoch 2/5
105/105 [==============================] - 0s 121us/sample - loss: 1.1470 - acc: 0.3048 - val_loss: 1.1232 - val_acc: 0.4000
Epoch 3/5
105/105 [==============================] - 0s 103us/sample - loss: 1.1442 - acc: 0.3048 - val_loss: 1.1217 - val_acc: 0.4000
Epoch 4/5
105/105 [==============================] - 0s 99us/sample - loss: 1.1392 - acc: 0.3048 - val_loss: 1.1207 - val_acc: 0.4000
Epoch 5/5
105/105 [==============================] - 0s 104us/sample - loss: 1.1358 - acc: 0.3048 - val_loss: 1.1192 - val_acc: 0.4000


In [0]:
pred = model.predict_classes(X_test)


### Report Accuracy of the predicted values

In [0]:
m = tf.keras.metrics.Accuracy()


In [57]:
m.update_state(y_test,pred)
print('Final result: ', m.result().numpy())  


Final result:  0.4
